**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
ls -1 ${FD_RES}/results/region

annotation_ccre
annotation_chipseq_histone
annotation_chipseq_tf
annotation_chipseq_tf_subset
annotation_chromHMM
annotation_crispri_hcrff
annotation_enhancer_junke
annotation_enhancer_zscore_junke
annotation_gencode
annotation_hic
annotation_tss_pol2
hic_intact_loop
KS91_K562_ASTARRseq_peak_macs_input
region_screened_crispri_flowfish.bed
region_screened_selected.bed
region_screened_selected_perbase.bed
region_screened_selected_perbase.bed.gz
region_screened_tmpra.bed
region_screened_tmpra_crispr.bed


In [3]:
ls -1 ${FD_RES}/results/region/annotation_chipseq_tf_subset

accession_info_peaks.tsv
description.tsv
download_log.txt
download.sh
K562.ENCSR000BMH.ENCFF398UQZ.YY1.bed.gz
K562.ENCSR000BMW.ENCFF707MDI.REST.bed.gz
K562.ENCSR000EFS.ENCFF190CGV.JUN.bed.gz
K562.ENCSR000EFT.ENCFF657CTC.GATA1.bed.gz
K562.ENCSR000EFU.ENCFF715WGN.ELK1.bed.gz
K562.ENCSR000EGE.ENCFF702XPO.EP300.bed.gz
K562.ENCSR000EGM.ENCFF660GHM.CTCF.bed.gz
K562.ENCSR000EHB.ENCFF852ZRK.TAL1.bed.gz
K562.ENCSR004GKA.ENCFF242AOL.ZEB2.bed.gz
K562.ENCSR091GVJ.ENCFF627RSK.ATF1.bed.gz
K562.ENCSR121SPB.ENCFF142ZTD.KLF10.bed.gz
K562.ENCSR388QZF.ENCFF355MNE.POLR2A.bed.gz
K562.ENCSR552YGL.ENCFF023IFO.NFE2.bed.gz
K562.ENCSR760UVO.ENCFF488OTN.KLF16.bed.gz
K562.ENCSR991ELG.ENCFF553GPK.SP1.bed.gz
K562.ENCSR998AJK.ENCFF777PKJ.NRF1.bed.gz


In [4]:
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf_subset
FP_INPS=($(ls ${FD_INP}/K562.*.bed.gz))

echo ${#FP_INPS[@]}
basename -a ${FP_INPS[@]}

16
K562.ENCSR000BMH.ENCFF398UQZ.YY1.bed.gz
K562.ENCSR000BMW.ENCFF707MDI.REST.bed.gz
K562.ENCSR000EFS.ENCFF190CGV.JUN.bed.gz
K562.ENCSR000EFT.ENCFF657CTC.GATA1.bed.gz
K562.ENCSR000EFU.ENCFF715WGN.ELK1.bed.gz
K562.ENCSR000EGE.ENCFF702XPO.EP300.bed.gz
K562.ENCSR000EGM.ENCFF660GHM.CTCF.bed.gz
K562.ENCSR000EHB.ENCFF852ZRK.TAL1.bed.gz
K562.ENCSR004GKA.ENCFF242AOL.ZEB2.bed.gz
K562.ENCSR091GVJ.ENCFF627RSK.ATF1.bed.gz
K562.ENCSR121SPB.ENCFF142ZTD.KLF10.bed.gz
K562.ENCSR388QZF.ENCFF355MNE.POLR2A.bed.gz
K562.ENCSR552YGL.ENCFF023IFO.NFE2.bed.gz
K562.ENCSR760UVO.ENCFF488OTN.KLF16.bed.gz
K562.ENCSR991ELG.ENCFF553GPK.SP1.bed.gz
K562.ENCSR998AJK.ENCFF777PKJ.NRF1.bed.gz


In [5]:
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf_subset
FN_INP=K562.ENCSR000BMH.ENCFF398UQZ.YY1.bed.gz
FP_INP=${FD_INP}/${FN_INP}

zcat ${FP_INP} | head

chr1	778486	778776	.	1000	.	132.48683	-1.00000	4.33080	136
chr1	904638	904883	.	1000	.	75.95987	-1.00000	4.33080	112
chr1	913032	913129	.	1000	.	41.88374	-1.00000	3.88398	56
chr1	921136	921238	.	1000	.	50.33578	-1.00000	4.33080	62
chr1	923707	924017	.	1000	.	34.69952	-1.00000	3.76640	155
chr1	924711	925021	.	634	.	20.27464	-1.00000	3.58222	155
chr1	939152	939462	.	1000	.	39.58505	-1.00000	3.91418	155
chr1	939368	939678	.	1000	.	17.19876	-1.00000	3.52219	155
chr1	940224	940534	.	686	.	24.18068	-1.00000	3.70828	155
chr1	941827	942137	.	1000	.	40.54679	-1.00000	3.90079	155


**RUN**

In [11]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf_subset.%a.txt \
    --array 0-15 \
    <<'EOF'
#!/bin/bash
### set directories
source ../config/config_duke.sh

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf_subset
FP_INPS=($(ls ${FD_INP}/K562.*.bed.gz))
FP_INP=${FP_INPS[SLURM_ARRAY_TASK_ID]}
FN_INP=$(basename ${FP_INP})

FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_REG=${FD_REG}/${FN_REG}

FD_OUT=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf_subset
FN_OUT=peak.annotation.${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
cat ${FP_REG} | head -5
echo
echo "Input: " ${FP_INP}
echo
echo "show first few lines of input"
zcat ${FP_INP} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -wo \
    -f 0.7 \
    -F 0.7 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29811005


**Check Results**

In [12]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf_subset.0.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         05-04-23+14:24:26

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf_subset/K562.ENCSR000BMH.ENCFF398UQZ.YY1.bed.gz

show first few lines of input
chr1	778486	778776	.	1000	.	132.48683	-1.00000	4.33080	136
chr1	904638	904883	.	1000	.	75.95987	-1.00000	4.33080	112
chr1	913032	913129	.	1000	.	41.88374	-1.00000	3.88398	56
chr1	921136	921238	.	1000	.	50.33578	-1.00000	4.33080	62
chr1	923707	924017	.	1000	.	34.69952	-1.00000	3.76640	155


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chipseq_tf

In [13]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf_subset.10.txt

Hostname:           x1-02-1.genome.duke.edu
Slurm Array Index:  10
Time Stamp:         05-04-23+14:24:26

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf_subset/K562.ENCSR121SPB.ENCFF142ZTD.KLF10.bed.gz

show first few lines of input
chr1	1609906	1610196	.	1000	.	54.00116	-1.00000	-0.46631	145
chr1	6603933	6604223	.	643	.	43.50786	-1.00000	-0.50695	145
chr1	8122997	8123287	.	599	.	42.19470	-1.00000	-0.50434	145
chr1	8869167	8869457	.	759	.	44.05824	-1.00000	-0.50172	145
chr1	8878287	8878577	.	678	.	48.26522	-1.00000	-0.48160	145


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annota

In [14]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_chipseq_tf_subset.15.txt

Hostname:           x1-02-2.genome.duke.edu
Slurm Array Index:  15
Time Stamp:         05-04-23+14:24:26

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf_subset/K562.ENCSR998AJK.ENCFF777PKJ.NRF1.bed.gz

show first few lines of input
chr1	115456	116012	.	652	.	43.47197	-1.00000	4.00600	278
chr1	605244	605809	.	1000	.	1170.12478	-1.00000	4.52697	219
chr1	778442	779304	.	1000	.	1273.41457	-1.00000	4.52697	343
chr1	778442	779304	.	1000	.	462.19284	-1.00000	4.52697	558
chr1	817373	817524	.	1000	.	155.55627	-1.00000	4.52697	9


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_chip